### **Let's explore Ray for an End-to-End Deep Learning Project!**

Welcome to an exploration of Ray, an open-source framework that empowers us to tackle the formidable challenges of training large deep learning models. You might be wondering, why Ray? Over the past four years, my curiosity has been fueled by the desire to understand how these massive models are trained. Think about it&mdash;training a vision transformer on your laptop with millions of parameters or even conducting ablation studies seems like an insurmountable task.

That's where Ray steps in. This remarkable framework offers distributed computing capabilities that enable us to train colossal models swiftly. It eliminates the need for expertise in infrastructure management, taking care of the heavy lifting. Moreover, transitioning from local development to a cloud environment is a breeze with Ray; no drastic code changes required. For an in-depth understanding of the framework, I urge you to refer to their [website](https://www.ray.io/). 


#### **Exploring this Repository**

Before we delve into the nitty-gritty details of this implementation, I'd like to extend my heartfelt gratitude to [Goku Mohandas](https://www.linkedin.com/in/goku/) for his outstanding [Made-With-ML](https://github.com/GokuMohandas/Made-With-ML) repository&mdash;a phenomenal source for MLOps; Their dedication to advancing the field and sharing knowledge has been a guiding light for countless enthusiasts, including myself. 

Now, let's briefly discuss the different modules applied in this implementation. 

1. **utils.py**: This module covers two functions focused on reading the configuration file (i.e., config.YAML) and setting the tracking URI for mlflow. 
2. **data_utils.py**: This module prepares the image classification data. This should be changed as required. Note that, in this case, we are working with Image Data. It covers two scenarios: when the data is stored locally and when the data needs to be downloaded. For simplicity, we can use the data from an online source.
3. **build_model.py**: This module defines the model for training. In this case, we are fine-tuning a ResNet50 Model. Therefore, it only initializes it with pre-trained weights. In this module, we can define custom Pytorch models as well.
4. **train_utils.py**: This module has functions that will be used in the training process.
5. **train_engine.py**: This is the module where we train the model using the Ray Framework. The critical concepts of Ray Train can be studied in this [documentation](https://docs.ray.io/en/latest/train/key-concepts.html). Trainers in Ray are required to perform distributed training. All we need to do is define a train_loop_per_worker function to perform the training steps that the different workers will use. To scale the training, we need to specify the Scaling Config. In addition, we must specify the Checkpoint config and Run Config that take care of the experiment's name, where the checkpoints will be stored, and any callbacks we are using. In this case, we will use mlflow to manage our experiments.
6. **tune_engine.py**: This module is responsible for performing the tuning. It utilizes the Ray's TorchTuner. Details can be gathered from [HERE](https://docs.ray.io/en/latest/tune/key-concepts.html). In addition to the TorchTrainer, Scaling Config, Run Config, and Checkpoint Config, we define the search space, search algorithms, and schedulars. The tuning is performed in a distributed manner, and the experiments can be tracked using mlflow.
7. **evaluate_engine.py**: This is the module for model evaluation as well as it can be adapted for batch inference.
8. **serve_engine.py**: Lastly, this implementation also explores model serving using Gradio with Ray. 

Also, the **config.YAML** file has to be updated, especially for the directories used in the project.

The code is in the form of Python scripts. To demonstrate how it works, in the notebook, these scripts are called using CLI commands. If you have any questions or comments, please do reach out!


---
**Handling Custom Datasets in Different Scenarios**

When you need to run the model on a custom dataset, you'll likely encounter two common scenarios: working with local data and working with remote data. Let's explore how to handle each scenario effectively:

**Scenario 1: Local Data**

Organizing Your Custom Dataset

If your custom dataset is stored locally, you can ensure a smooth integration by following these steps:

1. **Folder Structure**: Create a folder named `data` within your project directory. Inside this `data` folder, include subfolders for your custom dataset, such as `train`, `val` (validation), and `test`. Organize your data files accordingly.

2. **Data Loading**: Modify your data loading process in the `train_engine.py` module. Instead of importing the `build_datasets_download` method, import the `build_datasets_local` method. This adjustment allows you to read the custom dataset from the local directory structure.

**Scenario 2: Remote Data**

Accessing Data from a Remote Source

If your custom dataset is hosted remotely or available via a web service, you can access it as follows:

1. **Data Retrieval**: Utilize methods or libraries to fetch the data from the remote source. This could involve using APIs, downloading data from a web URL, or accessing cloud-based storage.

2. **Data Loading**: For the access of the data during training and tuning, you need to ensure that the correct function is imported from the 'data_utils.py' module. In this case, 'build_datasets_download' method needs to be imported.



---
**How to Use the `train_engine.py` Module**

Once you've tailored the configuration file to match your project's specifications, you can proceed with training the model using the `train_engine` module. This module can be executed via a command-line interface (CLI) command, simplifying the training process. Here's how to use it:

**CLI Command**

To initiate the training process, execute the following CLI command:

```bash
python train_engine.py


In [11]:
%run ../src/train_engine.py

(TorchTrainer pid=11541) Starting distributed worker processes: ['11605 (192.168.13.177)', '11606 (192.168.13.177)']
(RayTrainWorker pid=11605) Setting up process group for: env:// [rank=0, world_size=2]
(RayTrainWorker pid=11605) Moving model to device: cuda:0
(RayTrainWorker pid=11605) Wrapping provided model in DistributedDataParallel.


(RayTrainWorker pid=11606) Epoch 0-train Loss: 0.6301 Acc: 0.6803
(RayTrainWorker pid=11606) Epoch 2-train Loss: 0.3065 Acc: 0.9508 [repeated 8x across cluster]
(RayTrainWorker pid=11606) Epoch 4-train Loss: 0.1939 Acc: 0.9344 [repeated 8x across cluster]
(RayTrainWorker pid=11606) Epoch 6-train Loss: 0.1488 Acc: 0.9590 [repeated 8x across cluster]
(RayTrainWorker pid=11606) Epoch 8-train Loss: 0.1364 Acc: 0.9508 [repeated 8x across cluster]


2023-09-03 09:41:03,210	INFO tune.py:1148 -- Total run time: 43.81 seconds (43.76 seconds for the tuning loop).


Result(
  metrics={'val_loss': 0.14249894532718158, 'val_acc': 0.9736842105263158, 'should_checkpoint': True, 'done': True, 'trial_id': '6c48f_00000', 'experiment_tag': '0'},
  path='/home/dhaval/Projects/NewRay/results/ray_results/tuning-resnet-1693748416/TorchTrainer_6c48f_00000_0_2023-09-03_09-40-19',
  checkpoint=TorchCheckpoint(local_path=/home/dhaval/Projects/NewRay/results/ray_results/tuning-resnet-1693748416/TorchTrainer_6c48f_00000_0_2023-09-03_09-40-19/checkpoint_000009)
)


---
**How to Use the `tune_engine.py` Module**

After understanding how to execute the `train_engine`, let's explore the usage of the `tune_engine` module. This module allows you to perform multiple hyperparameter tuning runs based on the parameter space defined within `tune_engine.py`. In the future, consider configuring this parameter space in a separate configuration file to abstract away code details.

**Module Invocation**

The `tune_engine.py` module does not require any command-line arguments. Instead, it accesses the necessary parameters and configuration settings from a specified config file. The absence of command-line arguments simplifies usage and reduces the likelihood of errors.

**Configuration File**

To use the `tune_engine` effectively, make sure you have a configuration file in place. This file should contain the required arguments for your tuning runs.

**Hyperparameter Tuning**

The primary purpose of the `tune_engine` module is to perform hyperparameter tuning, where it systematically explores various combinations of hyperparameters to find the best configuration for your machine learning model. This process can help optimize your model's performance.




In [12]:
%run ../src/tune_engine.py

(TorchTrainer pid=16786) Starting distributed worker processes: ['16868 (192.168.13.177)', '16869 (192.168.13.177)']
(RayTrainWorker pid=16868) Setting up process group for: env:// [rank=0, world_size=2]
(RayTrainWorker pid=16868) Moving model to device: cuda:0
(RayTrainWorker pid=16868) Wrapping provided model in DistributedDataParallel.


(RayTrainWorker pid=16868) Epoch 0-train Loss: 0.6773 Acc: 0.6066
(RayTrainWorker pid=16868) Epoch 0-val Loss: 0.5774 Acc: 0.8026
(RayTrainWorker pid=16868) Epoch 1-train Loss: 0.5965 Acc: 0.6967
(RayTrainWorker pid=16868) Epoch 1-val Loss: 0.4606 Acc: 0.9211
(RayTrainWorker pid=16868) Epoch 2-train Loss: 0.4511 Acc: 0.8934
(RayTrainWorker pid=16868) Epoch 2-val Loss: 0.3749 Acc: 0.9342
(RayTrainWorker pid=16868) Epoch 3-train Loss: 0.3966 Acc: 0.9098
(RayTrainWorker pid=16868) Epoch 3-val Loss: 0.3245 Acc: 0.9211
(RayTrainWorker pid=16868) Epoch 4-train Loss: 0.3464 Acc: 0.9180
(RayTrainWorker pid=16868) Epoch 4-val Loss: 0.2703 Acc: 0.9474
(RayTrainWorker pid=16868) Epoch 5-train Loss: 0.2938 Acc: 0.9344
(RayTrainWorker pid=16868) Epoch 5-val Loss: 0.2443 Acc: 0.9605
(RayTrainWorker pid=16868) Epoch 6-train Loss: 0.2794 Acc: 0.9180
(RayTrainWorker pid=16868) Epoch 6-val Loss: 0.2237 Acc: 0.9605
(RayTrainWorker pid=16868) Epoch 7-train Loss: 0.2599 Acc: 0.9180
(RayTrainWorker pid=1686

(TorchTrainer pid=17337) Starting distributed worker processes: ['17413 (192.168.13.177)', '17414 (192.168.13.177)']
(RayTrainWorker pid=17413) Setting up process group for: env:// [rank=0, world_size=2]
(RayTrainWorker pid=17413) Moving model to device: cuda:0
(RayTrainWorker pid=17413) Wrapping provided model in DistributedDataParallel.


(RayTrainWorker pid=17413) Epoch 0-train Loss: 0.6903 Acc: 0.5328
(RayTrainWorker pid=17413) Epoch 0-val Loss: 0.6609 Acc: 0.6316


(TorchTrainer pid=17537) Starting distributed worker processes: ['17575 (192.168.13.177)', '17576 (192.168.13.177)']
(RayTrainWorker pid=17575) Setting up process group for: env:// [rank=0, world_size=2]
(RayTrainWorker pid=17575) Moving model to device: cuda:0
(RayTrainWorker pid=17575) Wrapping provided model in DistributedDataParallel.


...

(RayTrainWorker pid=21003) Epoch 0-train Loss: 0.7006 Acc: 0.5164
(RayTrainWorker pid=21003) Epoch 0-val Loss: 0.7488 Acc: 0.4737


(TorchTrainer pid=21156) Starting distributed worker processes: ['21197 (192.168.13.177)', '21198 (192.168.13.177)']
(RayTrainWorker pid=21197) Setting up process group for: env:// [rank=0, world_size=2]
(RayTrainWorker pid=21197) Moving model to device: cuda:0
(RayTrainWorker pid=21197) Wrapping provided model in DistributedDataParallel.


(RayTrainWorker pid=21197) Epoch 0-train Loss: 0.6474 Acc: 0.5656
(RayTrainWorker pid=21197) Epoch 0-val Loss: 0.4727 Acc: 0.7895
(RayTrainWorker pid=21197) Epoch 1-train Loss: 0.4349 Acc: 0.7951
(RayTrainWorker pid=21197) Epoch 1-val Loss: 0.1965 Acc: 0.9605
(RayTrainWorker pid=21197) Epoch 2-train Loss: 0.2349 Acc: 0.9016
(RayTrainWorker pid=21197) Epoch 2-val Loss: 0.1718 Acc: 0.9474
(RayTrainWorker pid=21197) Epoch 3-train Loss: 0.1468 Acc: 0.9590
(RayTrainWorker pid=21197) Epoch 3-val Loss: 0.1646 Acc: 0.9605


2023-09-03 10:01:52,206	INFO tune.py:1148 -- Total run time: 435.62 seconds (435.57 seconds for the tuning loop).


Result(
  metrics={'val_loss': 0.14027262243785357, 'val_acc': 0.9736842105263158, 'should_checkpoint': True, 'done': True, 'trial_id': '6b353_00014', 'experiment_tag': '14_batch_size=32,lr=0.0018'},
  path='/home/dhaval/Projects/NewRay/results/ray_results/tuning-resnet-1693749273/TorchTrainer_6b353_00014_14_batch_size=32,lr=0.0018_2023-09-03_09-54-36',
  checkpoint=TorchCheckpoint(local_path=/home/dhaval/Projects/NewRay/results/ray_results/tuning-resnet-1693749273/TorchTrainer_6b353_00014_14_batch_size=32,lr=0.0018_2023-09-03_09-54-36/checkpoint_000009)
)


**Evaluation Module**

This module is designed for evaluating the best model on a test/validation dataset. Here, we use the validation dataset, which is stored locally by default. If you need to evaluate the model on the test dataset, you will need to update the `data_utils.py` module accordingly.

**Usage**

To use this module, provide the name of the experiment as an argument when invoking the relevant function. The module will access the best run from the specified experiment and return the respective checkpoints for evaluation. The model loaded with the best checkpoint will be used to make predictions on the dataset.

Note: The results on the entire validation set and the one that you saw in the tuning runs will be different because we are checkpointing results only from one worker which is exposed to half of the entire dataset.


In [13]:
!python ../src/evaluate_engine.py --experiment-name "tuning-resnet-1693749273"

2023-09-03 10:02:37,510	WARNING torch_predictor.py:57 -- You have `use_gpu` as False but there are 2 GPUs detected on host where prediction will only use CPU. Please consider explicitly setting `TorchPredictor(use_gpu=True)` or `batch_predictor.predict(ds, num_gpus_per_worker=1)` to enable GPU prediction.
Accuracy:  0.954248366013072


**Serving Module**

The serving module is built using Gradio. This CLI command can be used to run the app.

In [ ]:
%run ../src/serve_engine.py --experiment_name "tuning-resnet-1693749273"